In [13]:
%load_ext autoreload
%autoreload 2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import least_squares
from scipy.interpolate import interp1d
from scipy.signal import periodogram, detrend

from structures import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
mach_1_data = read_mach_1_file(select_data_file())

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [17]:
map_data = get_subSurfaces(select_data_file(".map"), "QP_Mean")